# Курсовой проект «Введение в обработку естественного языка»

**Задание:**

Реализовать чат-бот на базе API Telegram.

**Интенты:**

1. болталка (разговорная часть): используются вопросы-ответы mail.ru,
2. суммаризация: используется предобученная модель - csebuetnlp/mT5_multilingual_XLSum
3. продуктовая часть: используются данные с youla.ru.

**Технологии:**

ML: CountVectorizer, TfidfVectorizer, FastText, MorphAnalyzer, dialogflow, LogisticRegression, annoy,
API: telegram


**Алгоритм работы чат-бота:**

При поступлении текстового запроса модель должна определить интент к которому относится запрос:

* болталка (вопрос/ответ)
* суммаризация
* поиск продукта

1. Если запрос "разговорный", используя TfidfVectorizer, FastText, annoy определяется наиболее подходящий ответ.

2. Если интент "cуммаризация", то используется предобученная модель csebuetnlp/mT5_multilingual_XLSum. Суммаризация должна начинаться с ключевого слова "Summarization:".

3. Если запрос "продуктовый", используя TfidfVectorizer, FastText, annoy (быстрый поиск ближайших соседей). Определяются N наиболее похожих продукта, которые возвращяются в чат.

4. Если в разговорном и продуктовом запросе найденный ответ слабо соответствует запросу, чат-бот должен ответить "Не понимаю запрос. Сформулируйте запрос более корректно.".``

## 1. Установка библиотек и подготовка данных.

In [1]:
!pip install telegram --quiet
!pip install python-telegram-bot --quiet
!pip install pymorphy2 --quiet
!pip install stop_words --quiet
!pip install annoy --quiet
!pip install transformers sentencepiece --quiet

!pip3 uninstall python-telegram-bot

!pip3 install python-telegram-bot

Found existing installation: python-telegram-bot 13.14
Uninstalling python-telegram-bot-13.14:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/python_telegram_bot-13.14.dist-info/*
    /usr/local/lib/python3.7/dist-packages/telegram/*
Proceed (y/n)? y
  Successfully uninstalled python-telegram-bot-13.14
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached python_telegram_bot-13.14-py3-none-any.whl (514 kB)


In [2]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip "/content/drive/MyDrive/06_lang/course_proj/Misis.Ida.zip"

Archive:  /content/drive/MyDrive/06_lang/course_proj/Misis.Ida.zip
  inflating: Misis. Ida/Otvety.txt   
error: invalid zip file with overlapped components (possible zip bomb)


## 2.Обучение разговорной модели

In [8]:
%%time
# работает быстро

assert True

#Small preprocess of the answers

question = None
written = False

c=0

# Идем по всем записям, берем строку как вопрос и после знака "---" находим ответ
with open("prepared_answers.txt", "w") as fout:
    with open("Misis. Ida/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


0it [00:00, ?it/s]

CPU times: user 20.9 s, sys: 2.12 s, total: 23 s
Wall time: 24.4 s


### Предобработка текста

In [9]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [10]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [11]:
assert True

# Preprocess for models fitting

sentences = []
c = 0

with open("Misis. Ida/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


0it [00:00, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
sentences = [i for i in sentences if len(i) > 2]

# записываем сериализованный объект в файл
with open(f'sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

In [ ]:
# # загружаем объект из файла
# with open(f'sentences.pkl', 'rb') as f:
#     sentences = pickle.load(f)

In [12]:
# загружаем объект из файла
with open(f'/content/drive/MyDrive/06_lang/course_proj/sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

### Обучение модели TF-IDF

In [13]:
# Функция-пустышка, для возможности передать в TfidfVectorizer список уже готовых токенов, а не текст
def simple_tokenizer(x):
    return x

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer=simple_tokenizer, lowercase=False, min_df=2)
tfidf_vectorizer.fit_transform(sentences)

idfs = {v[0]: v[1] for v in zip(tfidf_vectorizer.vocabulary_, tfidf_vectorizer.idf_)}
midf = np.mean(tfidf_vectorizer.idf_)

with open(f'idfs.pkl', 'wb') as f:
    pickle.dump(idfs, f)  # записывает сериализованный объект в файл.
    
with open(f'midf.pkl', 'wb') as f:
    pickle.dump(midf, f)  # записывает сериализованный объект в файл.

CPU times: user 8.34 s, sys: 184 ms, total: 8.53 s
Wall time: 8.63 s


In [16]:
with open(f'idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)  # загружает объект из файла

In [17]:
with open(f'midf.pkl', 'rb') as f:
    midf = pickle.load(f)  # загружает объект из файла

### Обучение модели FastText

In [18]:
SIZE_EMB = 200  # Размер эмбеддинга

In [ ]:
def embed_txt(txt, idfs, model, midf):
    n_ft = 0
    vector_ft = np.zeros(SIZE_EMB)
    for word in txt:
        if word in model:
            vector_ft += model[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
        
    return vector_ft

In [19]:
%%time
modelFT = FastText(sentences=sentences, size=SIZE_EMB, min_count=2, window=5, workers=8, seed=34)
modelFT.save("ft_model")

CPU times: user 12min 30s, sys: 7.36 s, total: 12min 37s
Wall time: 7min 9s


In [ ]:
%%time

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(SIZE_EMB ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = re.sub(r'\<[^>]*\>', '', spls[1]) # Удалим html-тэги
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(SIZE_EMB)

        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                # n_ft += 1
                n_ft += idfs.get(word, midf)

        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        if counter > 1000000:
            break

ft_index.build(10)
ft_index.save('ft_index.ann')

# Сохраняем индекс вопросов из болталки
with open(f'index_map.pkl', 'wb') as f:   # Save it for future use
    pickle.dump(index_map, f)  # записывает сериализованный объект в файл

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


CPU times: user 1h 14min 49s, sys: 1min 6s, total: 1h 15min 55s
Wall time: 1h 15min 8s


In [ ]:
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')  # Создание Annoy индекса
ft_index.load(f'ft_index.ann')  # загружает 

True

In [20]:
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')  # Создание Annoy индекса
ft_index.load(f'/content/drive/MyDrive/06_lang/course_proj/ft_index.ann')  # загружает 

True

In [ ]:
# with open(f'index_map.pkl', 'rb') as f:
#     index_map = pickle.load(f)  # загружает объект из файла

In [22]:

with open(f'/content/drive/MyDrive/06_lang/course_proj/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)  # загружает объект из файла

In [21]:
ft_index.get_nns_by_vector(np.zeros(SIZE_EMB), 2)

[17006, 25581]

## 3. Обучение продуктовых моделей

In [23]:
%%time

shop_data = pd.read_csv("/content/drive/MyDrive/06_lang/course_proj/ProductsDataset.csv")
# "/content/drive/MyDrive/06_lang/course_proj/Misis.Ida.zip"
shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

CPU times: user 2min 43s, sys: 751 ms, total: 2min 44s
Wall time: 2min 48s


,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [25]:
%%time

idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

CPU times: user 7min 44s, sys: 1.48 s, total: 7min 45s
Wall time: 7min 47s


In [26]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

Модуль CountVectorizer в sklearn позволяет сконвертировать набор текстов в матрицу токенов, находящихся в тексте.

In [28]:
%%time

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

CPU times: user 39 s, sys: 17.8 s, total: 56.7 s
Wall time: 37.1 s


In [29]:
with open(f'vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [30]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
with open(f'vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [31]:
with open(f'lr.pkl', 'wb') as f:
    pickle.dump(lr, f)

In [32]:
lr = LogisticRegression()
with open(f'lr.pkl', 'rb') as f:
    lr = pickle.load(f)

In [33]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9778754786555098

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

### Обучение модели TF-IDF

In [35]:
%%time
tfidf_vect_prod = TfidfVectorizer(lowercase=False, min_df=2)
tfidf_vect_prod.fit(X_train)

idfs_prod = {v[0]: v[1] for v in zip(tfidf_vect_prod.vocabulary_, tfidf_vect_prod.idf_)}
midf_prod = np.mean(tfidf_vect_prod.idf_)

with open(f'idfs_prod.pkl', 'wb') as f:
    pickle.dump(idfs_prod, f)
with open(f'midf_prod.pkl', 'wb') as f:
    pickle.dump(midf_prod, f)

CPU times: user 1.95 s, sys: 41.1 ms, total: 1.99 s
Wall time: 1.99 s


In [36]:
with open(f'idfs_prod.pkl', 'rb') as f:
    idfs_prod = pickle.load(f)

In [37]:
with open(f'midf_prod.pkl', 'rb') as f:
    midf_prod = pickle.load(f)

## Annoy. Алгоритм приблизительного поиска

In [38]:
%%time
ft_index_shop = annoy.AnnoyIndex(SIZE_EMB ,'angular')
index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(SIZE_EMB)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf_prod)
            n_ft += idfs.get(word, midf_prod)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(50)
# ft_index_shop.save('shop.ann')
ft_index_shop.save(f'ft_index_shop')

with open(f'index_map_shop.pkl', 'wb') as f:
    pickle.dump(index_map_shop, f)  # записывает сериализованный объект в файл.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/35548 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


CPU times: user 29.1 s, sys: 1.92 s, total: 31 s
Wall time: 27.3 s


In [40]:
ft_index_shop = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index_shop.load(f'ft_index_shop') 

True

In [41]:
with open(f'index_map_shop.pkl', 'rb') as f:
    index_map_shop = pickle.load(f)

# БОТ

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartTokenizer, MBartForConditionalGeneration

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [2]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_mT5_multilingual_XLSum = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [3]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import re

from telegram import Update
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

SIZE_EMB = 200  # Размер эмбеддинга

In [14]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

## Загрузка моделей

In [4]:
with open(f'idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)

with open(f'midf.pkl', 'rb') as f:
    midf = pickle.load(f)

with open(f'idfs_prod.pkl', 'rb') as f:
    idfs_prod = pickle.load(f)  # загружает объект из файла

with open(f'midf_prod.pkl', 'rb') as f:
    midf_prod = pickle.load(f)  # загружает объект из файла

# Подгружаем предварительно обученную модель вопросов из болталки
modelFT = FastText.load(f'ft_model')

# Подгружаем предварительно подготовленный и сохранненый индекс ответов из болталки
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index.load(f'/content/drive/MyDrive/06_lang/course_proj/ft_index.ann')
with open(f'/content/drive/MyDrive/06_lang/course_proj/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)  # загружает объект из файла

vectorizer = CountVectorizer(ngram_range=(1, 2))
with open(f'vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

lr = LogisticRegression()
with open(f'lr.pkl', 'rb') as f:
    lr = pickle.load(f)

with open(f'midf_prod.pkl', 'rb') as f:
    midf_p = pickle.load(f)

ft_index_shop = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index_shop.load(f'ft_index_shop') 

# Подгружаем предварительно обученную модель продуктовых названий
with open(f'index_map_shop.pkl', 'rb') as f:
    index_map_shop = pickle.load(f)

In [90]:
updater = Updater("5536499211:AAE23zQfPNhL1VkxtGE_LADiDl8ox05Hysc", use_context=True)  # Токен API к Telegram

def echo(update: Update, context: CallbackContext):
    txt = update.message.text
    update.message.reply_text('Ваше сообщение! ' + update.message.text)


def startCommand(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Добрый день!')


def model_mT5_multilingual_XLSum_summary(input_text, model, tokenizer): 
    """ 
        Суммаризация. Предобученную модель: csebuetnlp/mT5_multilingual_XLSum
    """
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(input_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary


def textMessage(update: Update, context: CallbackContext) -> None:
    
    # 1. Суммаризация. Начало фразы с ключевого слова "Summarization:"
    input_text = update.message.text
    if input_text.split(' ', 1)[0] == 'Summarisation:':
      update.message.reply_text('Подождите, идет суммаризация текста ...')

      input_text = input_text.replace('Summarisation:', '')
      summary = model_mT5_multilingual_XLSum_summary(input_text, model_mT5_multilingual_XLSum, tokenizer)
      update.message.reply_text('Суммаризация: ' + summary)

    else:
      res_text = 'Не понимаю запрос. Сформулируйте запрос более корректно.'
      input_txt = preprocess_txt(update.message.text)
      vect = vectorizer.transform([" ".join(input_txt)])
      prediction = lr.predict(vect)
      
      # 2. Определим является ли запрос продуктовым.
      if prediction[0] == 1:
          # Если запрос продуктовый, то найдём 3 самых подходящих товара.
          update.message.reply_text('Продуктовый ...')
          find = False
          vect_ft = embed_txt(input_txt, idfs_prod, modelFT, midf_prod)
          ft_index_shop_val, distances_shop = ft_index_shop.get_nns_by_vector(vect_ft, 3, include_distances=True)

          for i, item in enumerate(ft_index_shop_val):
              if distances_shop[i] <= 0.5:          
                title, image = index_map_shop[item]
                print(title, image)
                update.message.reply_text("title: {} image: {}".format(title, image))
                find = True
          if find == False:
            update.message.reply_text(res_text)

      else:
          # 3. Если запрос разговорный, то найдём ответ.
          update.message.reply_text('Разговорный ...')
          vect_ft = embed_txt(input_txt, idfs, modelFT, midf)
          ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
          if distances[0] <= 0.9:
              update.message.reply_text(index_map[ft_index_val[0]])
          else:
              update.message.reply_text(res_text)

In [91]:
dispatcher = updater.dispatcher  # Диспетчер

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler('start', startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start Bot
updater.start_polling()
updater.idle()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Автоматика для водяных насосов http://cache3.youla.io/files/images/360_360/59/e1/59e1f415c6ab9e3082768c53.jpg
Зонт-трость автомат новый, надежные спицы http://cache3.youla.io/files/images/360_360/5b/3e/5b3e3d89ec9855b45a1f29e2.jpg
Новые туфли Mascotte из натуральной кожи http://cache3.youla.io/files/images/360_360/5a/5d/5a5df19ecf204508ed201da2.jpg
